[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/ee-LandsatLinkr/blob/main/landsatlinkr_workshop_template.ipynb)

## **1. Environment Setup**

---

Import the Earth Engine API and authenticate to your EE account 
This step requires persistence :)
1. Run the block below, select Run Anyway if a pop up appears, then open the link that is generated
2. Choose a Cloud Project to use with this notebook.
If you don’t yet have a cloud-enabled project, select Choose Project > Create a New Cloud Project > and then give it a good name (this is permanent). 
You may have to then accept the Google Cloud Terms of Service. Open the link that is provided and select Agree and Continue on that page. 

3. Click 'Generate Token' (keep read-only scopes checked off)
4. Choose your EE account
5. Click Continue (not 'Back to Safety')
6. Check both of the checkboxes and select 'Continue'
7. Copy the Authorization Code at the bottom of the page and paste it in the code block below

*If you're already authenticated, you will see 'Successfully saved authorization token' instead of a box to paste the code into*



In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

Import the LandsatLinkr library from GitHub. This allows us to call the imported functions directly. 

In [ ]:
!rm -f -r /content/ee-LandsatLinkr
!git clone https://github.com/gee-community/ee-LandsatLinkr --quiet
import sys
sys.path.append("/content/ee-LandsatLinkr")
from landsatlinkr import *

## **2. Identity MSS WRS-1 Tile ID(s)**

---

First, we have to determine which WRS-1 tile we’ll be working with. WRS-1 tile IDs refer to the image footprints of the MSS sensor.

1. Open this EE script: https://code.earthengine.google.com/463f6aef9f6db3c248068bc0c6d9a530?hideCode=true 
2. In the map, zoom to your study area and select its location to reveal the WRS-1 tile IDs at that location (this may take a minute or two to load)
3. Copy the WRS-1 tile ID from the pop-up window and paste this WRS-1 tile ID into your run script as the **```WRS_1_GRANULE```** variable below

*If your study area overlaps with two or more tile footprints, you should note all of the WRS-1 tile IDs and process as many tiles that intersect the study region one at a time – the results can be composited later. For now, select one WRS-1 tile ID to begin with (you can only process one tile ID at a time).*

In [ ]:
WRS_1_GRANULE =  '047034'

## **3. Create Asset Folders**

---

The LandsatLinkr workflow currently exports assets to your Earth Engine account to store the intermediate and final processing results. LandsatLinkr therefore requires a specific folder structure to run and the function **`createProjectDir`** will create this structure once you have updated the **`PROJECT_DIR`** variable to indicate your assets folder. 


1. Create an asset folder called ‘LandsatLinkr’ to organize all of your LLR projects in one place (each tile ID will generate a new subfolder). Head to the Assets tab of the [EE Code Editor](https://code.earthengine.google.com/) to make this folder in your cloud project assets.

> You all now have at least one [cloud-enabled project](https://developers.google.com/earth-engine/cloud), so you’ll see two types of assets in your EE account: cloud project assets and legacy assets. Create your LandsatLinkr folder in your cloud project assets. Your new nested folders should look like this: 
- `projects/[cloud_project]/assets/LandsatLinkr`

2. Update the **`PROJECT_DIR`** variable to indicate **your** assets folder and run the code block below.

In [ ]:
# Cloud Project Asset Folder
# PROJECT_DIR = 'projects/[cloud_project]/assets/LandsatLinkr'

PROJECT_DIR = 'projects/ee-annalisertaylor/assets/LandsatLinkr'
createProjectDir(PROJECT_DIR, WRS_1_GRANULE)

## **4.1 Set Imagery Parameters**

---

These parameters determine which Landsat images are included in our analysis.
*   **CRS** controls the geographic projection of the outputs.
*   **DOY_RANGE** filters the imagery by certain days of the year, 1 through 365. The final output of LLR is annual composites, so this variable controls which parts of the year are included in the composites. 
*   **MAX_CLOUD** (0 to 100) controls the maximum allowable percentage of cloudiness for the included images.
*   **MAX_GEOM_RMSE** controls the maximum allowable spatial offset in units of a pixel.

In [ ]:
CRS = 'EPSG:3857'
DOY_RANGE = [160, 254]
MAX_CLOUD = 50
MAX_GEOM_RMSE = 0.5

# All of the parameters needed to run LandTrendr (final step)
LT_params = {
    # the bands you select here are the only ones you can then visualize as RGB
    # all choices: 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'ndvi',
    #   'tcb' (tasseled cap brightness), 'tcg' (tasseled cap greenness),
    #   'tcw' (tasseled cap wetness), and 'tca’ (tasseled cap angle).
    'ftvBands': ['red', 'green', 'blue'], 
    'maxSegments': 10,
    'spikeThreshold': 0.7,
    'vertexCountOvershoot': None,
    'preventOneYearRecovery': None,
    'recoveryThreshold': 0.5,
    'pvalThreshold': None,
    'bestModelProportion': None,
    'minObservationsNeeded': None,
    'scale': 30
}

# All of the parameters needed to run LandsatLinkr
LLR_params = {
  'maxRmseVerify': MAX_GEOM_RMSE,
  'maxCloudCover': MAX_CLOUD,
  'doyRange': DOY_RANGE,
  'wrs1': WRS_1_GRANULE,
  'crs': CRS,
  'excludeIds': [],
  'baseDir': PROJECT_DIR + '/' + WRS_1_GRANULE,
  'projectDir': PROJECT_DIR,
  'ltParams': LT_params,
  'correctOffset': True,
  'mssResample': 'nearest',
  'mssScale': 30
}

## **4.2 Preview and Filter MSS Images**

---


This step will print thumbnails of all of the available MSS images based on your parameters so that you can assess image quality and remove the bad images from your image collection. This step will result in an updated **`EXCLUDE_IDs`** variable, and can take between 10 to 30 minutes depending on the number of images available.

Preview MSS WRS-1 images, note bad images and add them to the **`EXCLUDE_IDS`** list. Run until all bad images are removed.

In [ ]:
# Uncomment these two lines to re-run with your bad image IDs removed
# EXCLUDE_IDS = []
# LLR_params['excludeIds'] = EXCLUDE_IDS

viewWrs1Col(LLR_params)

In [ ]:
EXCLUDE_IDS = [
    'LM10470341972208GDS03',
    'LM10470341972244AAA02',
    'LM10470341974161AAA02',
    'LM10470341974197AAA04',
    'LM20470341975201GDS03',
    'LM10470341976205AAA05',
    'LM10470341976223GDS03',
    'LM20470341976250GDS03',
    'LM20470341977190GDS04',
    'LM30470341978194GDS03',
    'LM20470341977226AAA05',
    'LM20470341977244GDS03',
    'LM20470341978185AAA02',
    'LM20470341978203AAA02',
    'LM30470341978212GDS03',
    'LM20470341978221AAA02',
    'LM20470341978239AAA02',
    'LM30470341979207AAA02',
    'LM20470341979216XXX01',
    'LM20470341980211AAA10',
    'LM30470341980220AAA03',
    'LM20470341980229AAA06',
    'LM20470341981205AAA03',
    'LM20470341981223AAA03',
    'LM30470341982173AAA08',
    'LM30470341982227AAA03'
]

LLR_params['excludeIds'] = EXCLUDE_IDS

## **5. Prepare the MSS Imagery**
---

**First**, we’ll create an MSS reference image. This reference image will be used for spectral normalization, as some of the sensors are inconsistent. This step will run a task that creates an image named 'ref' in your project's asset folder. This should take roughly 10 to 15 minutes. Once the task is completed (the word 'COMPLETED' will print below) and you see an image called 'ref' in your project's asset folder, you can move onto the next step.


In [ ]:
mssRefTask = exportMssRefImg(LLR_params)
monitorTaskStatus(mssRefTask)

**Next**, we’ll correct each included MSS image to that reference image **`ref`**. After running the block below, all of the selected MSS imagery will be harmonized within the MSS time series. This function runs a task to export a stack of annual composite images that are based on the mediod for each year, which is saved as **`MSS_WRS1_to_WRS2_stack`** in the asset folder that was created earlier. This step can take between 10 and 25 minutes. Once the task is completed, you can move onto the next step.


In [ ]:
mssWrs1ToWrs2Task = processMssWrs1Imgs(LLR_params)
monitorTaskStatus(mssWrs1ToWrs2Task)

## **6. Harmonize MSS Images to TM**

---

**First**, the function **`exportMss2TmCoefCol`** aggregates a  stratified random sample of pixels from coincident (collected simultaneously) MSS-TM image pairs. The resulting table (exported as an asset) is used to build a Random Forest (RF) regression model linking MSS imagery to TM imagery. This function will run a task that generates this table (**`mss_to_tm_coef_fc`**) and can take 10 to 20 minutes. Once this is completed you can move onto the next step.


In [ ]:
mss2TmInfoTask = exportMss2TmCoefCol(LLR_params)
monitorTaskStatus(mss2TmInfoTask)

**Next**, the function **`exportMssOffset`** will use coincident MSS and TM WRS-2 images to calculate a per-pixel offset image. This ensures that the MSS imagery more closely match the TM values at each pixel. This step will run one task (**`MSS_offset`**) that takes 15 to 30 minutes.

In [ ]:
mssOffsetTask = exportMssOffset(LLR_params)
monitorTaskStatus(mssOffsetTask)

**Finally**, the function **`exportFinalCorrectedMssCol`** will harmonize the WRS-1 MSS images to the TM time series using the Random Forest regression model calculated in a previous step. This step will run one task (**`WRS1_to_TM_stack`**) that takes 15 to 45 minutes.
*   Note that LandsatLinkr doesn't harmonize the WRS-2 (MSS) Landsat 4 and 5 images at all because there are coincident TM images which are preferred and used instead (except for 1983 when there are few TM images).

Once the task is completed, you can move onto the next step.

**We now have an MSS collection that is harmonized to TM!**

In [ ]:
mss2TmTask = exportFinalCorrectedMssCol(LLR_params)
monitorTaskStatus(mss2TmTask)

## **7. Run LandTrendr on the 50-year Time Series**

---

This step collects all of the Landsat imagery from later years (TM, ETM+, OLI, and OLI-2 sensors), filters them by the same days of year (**`DOY_RANGE`**), masks out cloudy pixels, and creates annual medoid composite images. This is combined with the MSS collection you exported in the previous step for a total of 50 years!

It then runs the [LandTrendr](https://emapr.github.io/LT-GEE/index.html) algorithm to distill 50+ years of spectral information into periods of relative stability and disturbance or change. This temporal segmentation information, such as the years and magnitude of disturbance, is captured in the output image, **`landtrendr`**, which is saved to your assets folder. This step can take approximately 2 hours to run.

For more information on how to interpret and work with this output in Earth Engine, check out this [guide](https://emapr.github.io/LT-GEE/lt-gee-outputs.html) or this [workshop](https://youtu.be/gsfHNzbmY10). 


In [ ]:
ltTask = exportLt(LLR_params)
monitorTaskStatus(ltTask)

## **8. Explore 50 Years of Landsat in Earth Engine**

---

Now that we’ve run all of our exports, we’ll move into the EE Code Editor to visualize and interactively inspect the results. Open [this example script](https://code.earthengine.google.com/?scriptPath=users%2Fannalisertaylor%2FG4G%3ALLR_Workshop_100622.js) in Earth Engine (and login if prompted). You may choose to save a copy to your account and name it ‘LLR_Workshop’ or something similar. Once your **`landtrendr`** asset has exported, change the main parameters in the EE script to investigate your data! 